In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import utils # stuff I've written for this project

import os, sys, datetime, glob, re
import os.path as op

import numpy as np
np.set_printoptions(precision=4)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import shutil

import nibabel as nib

import nilearn
from nilearn.masking import apply_mask
from nilearn.plotting import plot_img, plot_epi, plot_roi, plot_stat_map, view_img, plot_anat
from nilearn.image import load_img, math_img, resample_to_img, new_img_like
from nilearn.input_data import NiftiMasker

import nitime
import nitime.fmri.io as nfio
import nitime.timeseries as ts
import nitime.analysis as nta
import nitime.utils as ntu
import nitime.viz as ntv
import nitime.algorithms as ntalg

import scipy

from bids import BIDSLayout

In [3]:
%matplotlib inline

In [4]:
print(' numpy: ', np.__version__, '\n',
      'nibabel: ', nib.__version__, '\n',
      'nilearn: ', nilearn.__version__, '\n',
      'nitime: ', nitime.__version__, '\n',
      'scipy: ', scipy.__version__,)

 numpy:  1.20.0 
 nibabel:  3.2.1 
 nilearn:  0.7.0 
 nitime:  0.9 
 scipy:  1.6.0


## Initial definitions (before any preprocessing)

 * conn_ref_vol should be the first volume of the first run of the connectivity task
 * hemi_ref_vol should be the first volume of the first run of the hemifield task IN THE SAME SESSION
   * changed for this subject to have more explicit names for hemifield tasks from each session
     * lgn_hemi_ref_vol is the hemifield runs from the lgn session
 * lgn_ref_vol should be the reference volume of the LGN scans, a separate session (20191221 in this case)

In [5]:
sub = "LL"
ses_lgn = "20210511"
ses_prf = "20210810"
ses_conn = "20210909"

raw_data_dir = os.path.abspath("/Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/")
raw_layout = BIDSLayout(raw_data_dir, validate=False, derivatives=False)
derivs_dir = os.path.abspath("/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams")
out_dir = os.path.abspath(f"{derivs_dir}/sub-{sub}")
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
conn_ref_vol_name = f"sub-{sub}_ses-{ses_conn}_task-conn_refvol"
conn_ref_vol_path = os.path.abspath(f"{out_dir}/{conn_ref_vol_name}.nii.gz")
conn_mask_file = os.path.join(out_dir, f"sub-{sub}_ses-{ses_conn}_task-conn_mask.nii.gz")
lgn_hemi_ref_vol_name = f"sub-{sub}_ses-{ses_lgn}_task-hemi_refvol"
lgn_hemi_ref_vol_path = os.path.abspath(f"{out_dir}/{lgn_hemi_ref_vol_name}.nii.gz")
lgn_hemi_mask_file = os.path.join(out_dir, f"sub-{sub}_ses-{ses_lgn}_task-hemi_mask.nii.gz")
prf_ref_vol_name = f"sub-{sub}_ses-{ses_prf}_task-prf_refvol.nii.gz"
prf_ref_vol_path = os.path.abspath(f"{out_dir}/{prf_ref_vol_name}")
lgn_hemi_ref_vol_img = load_img(lgn_hemi_ref_vol_path) # will fail if refvol not created yet
prf_ref_vol_img = load_img(prf_ref_vol_path) # will fail if refvol not created yet
conn_ref_vol_img = load_img(conn_ref_vol_path) # will fail if refvol not created yet

//anaconda/envs/streams/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


## Begin Preprocessing

### Make ref_vol from first dicom

In [ ]:
### need to escape spaces!
### Do for each sequence
first_dicom_hemi = "/Users/smerdis/data/LGN/Ll_Lgn_20210511/Silver_Arjun\ -\ 1/sb_bold_1p75mmISO_PSN_3/IM-0003-0001.dcm"
first_dicom_prf = "/Users/smerdis/data/LGN/Nb_20191221/Silver_Arjun\ -\ 1/sb_bold_1p75mmISO_PSN_7/IM-0007-0001.dcm"
first_dicom_conn = "/Users/smerdis/data/LGN/Ll_Mpconn_20210909/Silver_Arjun\ -\ 1/mb_bold_mb3_1p75mm_PSN_1p75slice_4/IM-0004-0001.dcm"

In [ ]:
!ls {first_dicom_conn}
#!ls {first_dicom_hemi}
#!ls {first_dicom_prf}

In [ ]:
dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {conn_ref_vol_name} -z y -o {out_dir} {first_dicom_conn}"
# dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {prf_ref_vol_name} -z y -o {out_dir} {first_dicom_prf}"
#dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {lgn_hemi_ref_vol_name} -z y -o {out_dir} {first_dicom_hemi}"
print(dcm2niix_cmd)
os.system(dcm2niix_cmd)

In [ ]:
# dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {lgn_hemi_ref_vol_name} -z y -o {out_dir} {first_dicom_hemi}"
# print(dcm2niix_cmd)

In [ ]:
!ls {out_dir}

### Define refence volume

In [ ]:
#conn_ref_vol_img = load_img(conn_ref_vol_path)
# lgn_hemi_ref_vol_img = load_img(lgn_hemi_ref_vol_path)
# because hemifield and mp use the same sequence and are collected in the same session, just use the same refvol
# mp_ref_vol_img = lgn_hemi_ref_vol_img
#prf_ref_vol_img = load_img(prf_ref_vol_path)

In [ ]:
raw_bolds_conn = sorted([f for f in raw_layout.get(subject=sub, session=ses_conn, task='conn', suffix='bold',
            extension=['nii.gz'], return_type='file')])
raw_bolds_hemi = sorted([f for f in raw_layout.get(subject=sub, session=ses_lgn, task='hemi', suffix='bold',
            extension=['nii.gz'], return_type='file')])
raw_bolds_mp = sorted([f for f in raw_layout.get(subject=sub, session=ses_lgn, task='mp', suffix='bold',
            extension=['nii.gz'], return_type='file')])

In [ ]:
!ls {lgn_hemi_ref_vol_path}

In [ ]:
print(raw_bolds_hemi, raw_bolds_conn)

### mcflirt the bold files using ref_vol as the reference image, save them

In [ ]:
print("\n".join(raw_bolds_conn))

In [ ]:
# MPConn
for this_epi in raw_bolds_conn:
    align_to = 'refvol' #'refvol' # or 'anat'
    epi_stub_mcf = utils.change_bids_description(this_epi, 'desc-mcflirt')   
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses_conn}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses_conn}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses_conn}", "func", epi_stub_mcf)
    if align_to == 'refvol':
        mcflirt_cmd = f"mcflirt -reffile {conn_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    elif align_to == 'anat':
        mcflirt_cmd = f"mcflirt -reffile {anat_file} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}_anatalign"
    print(mcflirt_cmd)
    #os.system(mcflirt_cmd)

In [ ]:
# LGN session hemifield
for this_epi in raw_bolds_hemi:
    align_to = 'refvol' #'refvol' # or 'anat'
    epi_stub_mcf = utils.change_bids_description(this_epi, 'desc-mcflirt')   
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses_lgn}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses_lgn}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses_lgn}", "func", epi_stub_mcf)
    if align_to == 'refvol':
        mcflirt_cmd = f"mcflirt -reffile {lgn_hemi_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    elif align_to == 'anat':
        mcflirt_cmd = f"mcflirt -reffile {anat_file} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}_anatalign"
    print(mcflirt_cmd)
    os.system(mcflirt_cmd)

In [ ]:
# LGN session MPLocalizer
for this_epi in raw_bolds_mp:
    align_to = 'refvol' #'refvol' # or 'anat'
    epi_stub_mcf = utils.change_bids_description(this_epi, 'desc-mcflirt')   
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses_lgn}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses_lgn}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses_lgn}", "func", epi_stub_mcf)
    if align_to == 'refvol':
        mcflirt_cmd = f"mcflirt -reffile {lgn_hemi_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    elif align_to == 'anat':
        mcflirt_cmd = f"mcflirt -reffile {anat_file} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}_anatalign"
    print(mcflirt_cmd)
    os.system(mcflirt_cmd)

In [ ]:
!ls /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/ses-20210909/func/sub-LL_ses-20210909_task-conn_run-02_desc-mcflirt_bold.nii.gz

### Make brainmask if it doesn't exist already

In [ ]:
conn_ref_bold_run = raw_bolds_conn[0]
conn_TR = raw_layout.get_metadata(conn_ref_bold_run)['RepetitionTime'] # instead of get_tr()

In [ ]:
conn_TR

In [ ]:
!ls {conn_mask_file}

In [ ]:
conn_masker = NiftiMasker(detrend=False, standardize=False, mask_strategy="epi", t_r=conn_TR)
conn_masker.fit(conn_ref_bold_run)
conn_masker.mask_img_.to_filename(conn_mask_file)
conn_mask_img = conn_masker.mask_img_
print(conn_mask_file, conn_mask_img.shape)

In [ ]:
hemi_ref_bold_run = raw_bolds_hemi[0]
print(hemi_ref_bold_run)
hemi_TR = raw_layout.get_metadata(task='hemi', session=ses_lgn)['RepetitionTime']
hemi_masker = NiftiMasker(detrend=False, standardize=False, mask_strategy="epi", t_r=hemi_TR)
hemi_masker.fit(hemi_ref_bold_run)
hemi_masker.mask_img_.to_filename(lgn_hemi_mask_file)
hemi_mask_img = hemi_masker.mask_img_
print(lgn_hemi_mask_file, hemi_mask_img.shape)

### Move event and json files

In [ ]:
!ls "{raw_data_dir}/ses-{ses_lgn}/func/"

In [ ]:
events_glob = f"{raw_data_dir}/ses-{ses_lgn}/func/*events*.tsv"
for ef in glob.glob(events_glob):
    parts = ef.split('_')
    for i, part in enumerate(parts):
        #print(i, part)
        if 'sub-js' in part:
            parts[i] = part.replace('sub-js', 'sub-JS')
    [lastpart, ext] = parts[-1].split('.')
    print(parts, lastpart, ext, lastpart.isnumeric())
    if lastpart.isnumeric(): # the event file has a suffix indicating the time it was written, remove it
        parts[-2] = parts[-2] + '.' + ext
        out_fn = '_'.join(parts[:-1])
        cmd = f"cp {ef} {out_fn}"
        print(cmd)
        os.system(cmd)
task_json_glob = f"{raw_data_dir}/ses-{ses_lgn}/func/*task*json"
print("\n".join(glob.glob(task_json_glob)))

## Define paths after preprocessing is complete

In [6]:
preproc_layout = BIDSLayout(out_dir, validate=False)
# get only the big LGN masks, not any smaller M/P rois already assigned
rois = [f for f in
        preproc_layout.get(subject=sub, extension=['.nii.gz'], suffix='roi', return_type='file', space='lgn')
        if ("desc-LLGN" in f or "desc-RLGN" in f) and ('T1w' not in f)]
big_rois = [f for f in
        preproc_layout.get(subject=sub, extension=['.nii.gz'], suffix='roi', return_type='file', space='lgn')
        if ("desc-LLGN" in f or "desc-RLGN" in f) and ('P' not in f and 'M' not in f and 'T1w' not in f)]
cortical_rois = [f for f in
        preproc_layout.get(subject=sub, extension=['.nii.gz'], suffix='roi', return_type='file')
        if ("desc-Lcortex" in f or "desc-Rcortex" in f)]
lgn_ref_vol = lgn_hemi_ref_vol_path # using hemifield ref vol as refvol for all lgn scans (ses 20210505)
lgn_ref_img = load_img(lgn_ref_vol)
lgn_mask_file = lgn_hemi_mask_file # and hemifield mask as mask for all lgn scans (ses 20210505)
#lgn_coh_fn = os.path.join(out_dir, f"sub-{sub}_ses-20191221_task-hemi_desc-LGNcoherence_map.nii")
#hemi_coh_fn = os.path.join(out_dir, f"sub-{sub}_ses-{ses}_task-hemi_desc-LGNcoherence_map.nii")
#cortex_coh_fn = os.path.join(out_dir, f"sub-{sub}_ses-20191221_task-hemi_desc-cortexcoherence_map.nii")
anat_file = raw_layout.get(subject=sub, session=ses_lgn, extension=['nii.gz'], suffix="T1w", return_type='file')[0]
anat_img = load_img(anat_file)

print("LGN rois (including M/P):", '\n'.join(rois),
      "LGN rois (LGN only):", '\n'.join(big_rois),
      "Cortical rois: ", '\n'.join(cortical_rois),
      "Reference BOLD volume for connectivity:", conn_ref_vol_path,
      "Brainmask for connectivity", conn_mask_file,
#      "Reference BOLD volume for hemifield scans in connectivity session:", hemi_ref_vol_path,
#      "Brainmask for hemifield scans in connectivity session", hemi_mask_file,
      "LGN session refvol: ", lgn_ref_vol,
      "LGN session brainmask: ", lgn_mask_file,
      "anat file: ", anat_file, sep="\n")

LGN rois (including M/P):
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGNM80_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGNP80_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGNM80_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGNP80_space-lgn_roi.nii.gz
LGN rois (LGN only):
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-lgn_roi.nii.gz
/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_space-lgn_

In [7]:
print(f"fsleyes {anat_file} {lgn_ref_vol} {prf_ref_vol_path} {conn_ref_vol_path}")

fsleyes /Users/smerdis/data/LGN/BIDS/LL_combined/sub-LL/ses-20210511/anat/sub-LL_ses-20210511_T1w.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_task-hemi_refvol.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210810_task-prf_refvol.nii.gz /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210909_task-conn_refvol.nii.gz


## Registration and Resampling

In [8]:
fslmaths_roi_threshold = 0.5

conn_to_t1_xfm_path = f"{out_dir}/sub-{sub}_from-conn_to-T1w_mode-image_xfm.txt"
t1_to_conn_xfm_path = f"{out_dir}/sub-{sub}_from-T1w_to-conn_mode-image_xfm.txt"

lgn_to_conn_xfm_path = f"{out_dir}/sub-{sub}_from-lgn_to-conn_mode-image_xfm.txt"
conn_to_lgn_xfm_path = f"{out_dir}/sub-{sub}_from-conn_to-lgn_mode-image_xfm.txt"

lgn_to_t1_xfm_path = f"{out_dir}/sub-{sub}_from-lgn_to-T1w_mode-image_xfm.txt"
t1_to_lgn_xfm_path = f"{out_dir}/sub-{sub}_from-T1w_to-lgn_mode-image_xfm.txt"

prf_to_conn_xfm_path = f"{out_dir}/sub-{sub}_from-prf_to-conn_mode-image_xfm.txt"
conn_to_prf_xfm_path = f"{out_dir}/sub-{sub}_from-conn_to-prf_mode-image_xfm.txt"

prf_to_t1_xfm_path = f"{out_dir}/sub-{sub}_from-prf_to-T1w_mode-image_xfm.txt"
t1_to_prf_xfm_path = f"{out_dir}/sub-{sub}_from-T1w_to-prf_mode-image_xfm.txt"

prf_to_lgn_xfm_path = f"{out_dir}/sub-{sub}_from-prf_to-lgn_mode-image_xfm.txt"
lgn_to_prf_xfm_path = f"{out_dir}/sub-{sub}_from-lgn_to-prf_mode-image_xfm.txt"

In [9]:
# brain extract the T1 for better registration and save it as _brain in out_dir
fn_parts = op.basename(anat_file).split('.')
anat_file_bet_stub = '.'.join([f"{fn_parts[0]}_brain", *fn_parts[1:]])
anat_file_bet = f"{out_dir}/{anat_file_bet_stub}"
# bet_cmd = f"bet {anat_file} {anat_file_bet}"
# print(anat_file, anat_file_bet, bet_cmd, sep="\n")
# print(os.system(bet_cmd))

#### Register (FLIRT) first MB3 (MPConn) epi to T1, save transform, inverse, and resampled refvol

In [10]:
flirt_cmd = f"flirt -in {conn_ref_vol_path} -ref {anat_file_bet} -omat {conn_to_t1_xfm_path} -out {out_dir}/sub-{sub}_ses-{ses_conn}_task-conn_refvol_space-T1w.nii.gz"
print(flirt_cmd)
os.system(flirt_cmd)

inverse_cmd = f"convert_xfm -omat {t1_to_conn_xfm_path} -inverse {conn_to_t1_xfm_path}"
os.system(inverse_cmd)

flirt -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210909_task-conn_refvol.nii.gz -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_T1w_brain.nii.gz -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-conn_to-T1w_mode-image_xfm.txt -out /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210909_task-conn_refvol_space-T1w.nii.gz


0

#### FLIRT LGN MP localizer to MPConn (MB3), save transform, inverse, and resampled refvol

In [11]:
flirt_cmd = f"flirt -in {lgn_ref_vol} -ref {conn_ref_vol_path} -omat {lgn_to_conn_xfm_path} "
flirt_cmd = flirt_cmd + f"-out {out_dir}/sub-{sub}_ses-{ses_lgn}_task-lgn_refvol_space-conn.nii.gz"
print(flirt_cmd)
os.system(flirt_cmd)

inverse_cmd = f"convert_xfm -omat {conn_to_lgn_xfm_path} -inverse {lgn_to_conn_xfm_path}"
os.system(inverse_cmd)

flirt -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_task-hemi_refvol.nii.gz -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210909_task-conn_refvol.nii.gz -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-conn_mode-image_xfm.txt -out /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_task-lgn_refvol_space-conn.nii.gz


0

#### FLIRT prf <--> conn (MB3)

In [12]:
flirt_cmd = f"flirt -in {prf_ref_vol_path} -ref {conn_ref_vol_path} -omat {prf_to_conn_xfm_path} -out {out_dir}/sub-{sub}_ses-{ses_prf}_task-prf_refvol_space-conn.nii.gz"
print(flirt_cmd, os.system(flirt_cmd))

inverse_cmd = f"convert_xfm -omat {conn_to_prf_xfm_path} -inverse {prf_to_conn_xfm_path}"
print(inverse_cmd, os.system(inverse_cmd))

flirt -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210810_task-prf_refvol.nii.gz -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210909_task-conn_refvol.nii.gz -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-prf_to-conn_mode-image_xfm.txt -out /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210810_task-prf_refvol_space-conn.nii.gz 0
convert_xfm -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-conn_to-prf_mode-image_xfm.txt -inverse /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-prf_to-conn_mode-image_xfm.txt 0


#### Concat LGN ---> MB3 (Conn) ----> T1

In [13]:
# convert_xfm -omat AtoC.mat -concat BtoC.mat AtoB.mat
concat_cmd = f"convert_xfm -omat {lgn_to_t1_xfm_path} -concat {conn_to_t1_xfm_path} {lgn_to_conn_xfm_path}"
print(concat_cmd)
os.system(concat_cmd)

convert_xfm -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-T1w_mode-image_xfm.txt -concat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-conn_to-T1w_mode-image_xfm.txt /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-conn_mode-image_xfm.txt


0

#### Concat (pRF ---> MB3 (Conn) ---> T1) and invert this

In [16]:
concat_cmd = f"convert_xfm -omat {prf_to_t1_xfm_path} -concat {conn_to_t1_xfm_path} {prf_to_conn_xfm_path}"
print(concat_cmd, os.system(concat_cmd))

inverse_cmd = f"convert_xfm -omat {t1_to_prf_xfm_path} -inverse {prf_to_t1_xfm_path}"
print(inverse_cmd, os.system(inverse_cmd))

convert_xfm -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-prf_to-T1w_mode-image_xfm.txt -concat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-conn_to-T1w_mode-image_xfm.txt /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-prf_to-conn_mode-image_xfm.txt 0
convert_xfm -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-T1w_to-prf_mode-image_xfm.txt -inverse /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-prf_to-T1w_mode-image_xfm.txt 0


#### Concat ((LGN ---> MB3 (Conn) ---> T1) ---> pRF)

In [17]:
# convert_xfm -omat AtoC.mat -concat BtoC.mat AtoB.mat
concat_cmd = f"convert_xfm -omat {lgn_to_prf_xfm_path} -concat {t1_to_prf_xfm_path} {lgn_to_t1_xfm_path}"
print(concat_cmd)
os.system(concat_cmd)

convert_xfm -omat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-prf_mode-image_xfm.txt -concat /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-T1w_to-prf_mode-image_xfm.txt /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-T1w_mode-image_xfm.txt


0

#### FLIRT ROIs from LGN func space to T1

In [18]:
for roi in big_rois:
    resampled_roi_fn = op.join(out_dir, utils.change_bids_description(roi, 'space-T1w', 'space-'))
    roi_flirt_cmd = f"flirt -ref {anat_file_bet} -in {roi} -applyxfm -init {lgn_to_t1_xfm_path} -out\
                {resampled_roi_fn}"
    print(roi_flirt_cmd, os.system(roi_flirt_cmd))

flirt -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_T1w_brain.nii.gz -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-lgn_roi.nii.gz -applyxfm -init /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-T1w_mode-image_xfm.txt -out                /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-T1w_roi 0
flirt -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_T1w_brain.nii.gz -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_space-lgn_roi.nii.gz -applyxfm -init /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-T1w_mode-image_xfm.txt -out                /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_space-T1w_roi 0


#### Now FLIRT the ROIs (from their native LGN EPI space) to pRF space and binarize

In [28]:
for roi in big_rois:
    resampled_roi_fn = f"{out_dir}/{utils.change_bids_description(roi, 'space-prf', 'space')}"
    roi_flirt_cmd = (f"flirt -ref {prf_ref_vol_path} -in {roi} -applyxfm "
        f"-init {lgn_to_prf_xfm_path} -out {resampled_roi_fn} "
        f"&& fslmaths {resampled_roi_fn} -thr {fslmaths_roi_threshold} -bin {resampled_roi_fn}")
    print(roi_flirt_cmd, os.system(roi_flirt_cmd))

flirt -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210810_task-prf_refvol.nii.gz -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-lgn_roi.nii.gz -applyxfm -init /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_from-lgn_to-prf_mode-image_xfm.txt -out /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-prf_roi && fslmaths /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-prf_roi -thr 0.5 -bin /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-LLGN_space-prf_roi 0
flirt -ref /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210810_task-prf_refvol.nii.gz -in /Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/streams/sub-LL/sub-LL_ses-20210511_desc-RLGN_space-lgn_roi.nii.gz -applyxfm -init

#### resample ROIs using nilearn

In [ ]:
# resampled_rois = []
# for roi in final_big_rois:
#     roi_img = load_img(roi)
#     print(roi_img.shape, roi_img.affine, sep='\n')
#     roi_fn_parts = roi.split('_')
#     roi_fn_parts.insert(-1, 'space-T1w')
#     resampled_roi_fn = '_'.join(roi_fn_parts)
#     resampled_roi_img = resample_to_img(roi_img, anat_img, 'nearest')
#     resampled_roi_img.to_filename(resampled_roi_fn)
#     resampled_rois.append(resampled_roi_fn)
#     print(resampled_roi_img.shape, resampled_roi_img.affine, sep='\n')
#     plot_roi(resampled_roi_img, bg_img=anat_img)

In [ ]:
# try:
#     if resampled_rois:
#         pass
#     else:
#         resampled_rois = [f for f in
#         preproc_layout.get(subject=sub, extension=['.nii.gz'], suffix='roi', return_type='file')
#         if ("desc-LLGN" in f or "desc-RLGN" in f) and ('T1w_nilearn' in f)]
# except NameError:
#     resampled_rois = [f for f in
#         preproc_layout.get(subject=sub, extension=['.nii.gz'], suffix='roi', return_type='file')
#         if ("desc-LLGN" in f or "desc-RLGN" in f) and ('T1w_nilearn' in f)]
# print('\n'.join(resampled_rois))

### Write hemifield event files (they are all identical)

In [ ]:
utils.write_hemifield_localizer_event_file(f"{raw_data_dir}/ses-{ses_lgn}/func/sub-{sub}_ses-{ses_lgn}_task-hemi_run-01_events.tsv")
utils.write_hemifield_localizer_event_file(f"{raw_data_dir}/ses-{ses_lgn}/func/sub-{sub}_ses-{ses_lgn}_task-hemi_run-02_events.tsv")

In [ ]:
for e in glob.glob(events_glob):
    fileparts = op.split(e)[-1].split('_')[:-1]
    fileparts[0] = f"sub-{sub}"
    #runpart = fileparts[-2]
    #print('_'.join(fileparts), fileparts[-2])
    fn = '_'.join(fileparts)
    new_file_name = f"{raw_data_dir}/ses-{ses}/func/{fn}.tsv"
    print(e, new_file_name, sep="\n")
    #shutil.copyfile(e, new_file_name)

In [ ]:
for e in glob.glob(task_json_glob):
    fn = os.path.split(e)[-1]
    new_file_name = f"{raw_data_dir}/ses-{ses}/func/{fn}"
    print(e, new_file_name, sep="\n")
    #shutil.copyfile(e, new_file_name)

In [ ]:
blocks_in_order = ['L', 'R']
for thisrun in [1]:
    events_fn = f"{raw_data_dir}/ses-{ses}/func/sub-{sub}_ses-{ses}_task-hemi_run-{thisrun:02d}_events.tsv"
    events_file_contents = f"onset\tduration\ttrial_type\n"
    time_between_onsets = 13.5
    for i in range(22):
        events_file_contents += f"{i*time_between_onsets:.2f}\t{time_between_onsets}\t{blocks_in_order[i%(len(blocks_in_order))]}\n"
    #with open(events_fn, 'w') as f:
        #f.write(events_file_contents)
    print(events_fn, '\n', events_file_contents)

## GLM invocation from a cell

### Individual GLMs

Run the GLMs for each task using all runs

In [ ]:
%%time
hemi_workdir = utils.run_fixedeffects_glm(sub, ses_lgn, "hemi", [1, 2], raw_data_dir, out_dir)

In [ ]:
hemi_datasink = f"{hemi_workdir}/fixedeffects/modelfit/datasink"
print(hemi_workdir, hemi_datasink, sep="\n")
!ls {hemi_datasink}
hemi_RL_l1, hemi_RL_l2 = utils.get_model_outputs(hemi_datasink, [1])
hemi_LR_l1, hemi_LR_l2 = utils.get_model_outputs(hemi_datasink, [2])
beta_RL = hemi_RL_l2[0]
beta_LR = hemi_LR_l2[0]
print(beta_RL)

In [ ]:
print(f"fsleyes {lgn_hemi_ref_vol_path} {beta_RL}")

In [ ]:
%%time
mp_workdir = utils.run_fixedeffects_glm(sub, ses_lgn, "mp", [1, 2, 3, 4, 5, 6, 7, 8], raw_data_dir, out_dir,
                                       working_dir_suffix='20210825')

In [ ]:
mp_datasink = f"{mp_workdir}/fixedeffects/modelfit/datasink"
print(mp_workdir, mp_datasink, sep="\n")
!ls {mp_datasink}

In [ ]:
mp_l1, mp_l2 = utils.get_model_outputs(mp_datasink, [1])
pm_l1, pm_l2 = utils.get_model_outputs(mp_datasink, [2])


In [ ]:
beta_MP = mp_l2[0]
beta_PM = pm_l2[0]
beta_MP_img = load_img(beta_MP)

In [ ]:
print(beta_RL, beta_LR, beta_MP, sep='\n')

In [ ]:
beta_RL_fn = f"{out_dir}/sub-{sub}_ses-{ses_lgn}_task-hemi_desc-betaRL_space-lgn_map.nii.gz"
beta_LR_fn = f"{out_dir}/sub-{sub}_ses-{ses_lgn}_task-hemi_desc-betaLR_space-lgn_map.nii.gz"
os.system(f"cp {beta_RL} {beta_RL_fn}")
os.system(f"cp {beta_LR} {beta_LR_fn}")

In [ ]:
beta_MP_fn = f"{out_dir}/sub-{sub}_ses-{ses_lgn}_task-mp_desc-betaMP_space-lgn_map.nii.gz"
os.system((f"cp {beta_MP} {beta_MP_fn}"))

In [ ]:
print(f"fsleyes {anat_file} {lgn_ref_vol} {beta_RL_fn} {beta_LR_fn} {beta_MP_fn}")

### Grab the affine transformation matrix so we can convert EPI to scanner space

In [ ]:
# epi_vox_center = (np.array(ref_vol_img.get_fdata().shape) - 1) / 2.
# print('epi vox center: ', epi_vox_center)
# print(ref_vol_path)

### Assign M/P rois based on GLM Beta maps

In [ ]:
print('\n'.join(rois))

In [ ]:
print(lgn_ref_vol)

In [ ]:
for roi in rois:
    utils.roi_stats({roi:load_img(roi)}, load_img(lgn_ref_vol))

In [ ]:
big_rois

In [ ]:
final_big_rois = big_rois # select the ones you want to subdivide

In [ ]:
for roi in final_big_rois:
    utils.roi_stats({roi:load_img(roi)}, lgn_ref_img)
    if 'desc-L' in roi:
        M_roi, P_roi, threshold = utils.assign_roi_percentile(roi, beta_MP_img, 80, lgn_ref_img, 'L')
    elif 'desc-R' in roi:
        M_roi, P_roi, threshold = utils.assign_roi_percentile(roi, beta_MP_img, 80, lgn_ref_img, 'R')
    else:
        print('Error - not sure which hemisphere roi is in')
    utils.roi_stats({'M':load_img(M_roi), 'P':load_img(P_roi)}, lgn_ref_img)

In [ ]:
for roi in final_big_rois:
    parts = op.basename(roi).split('_')
    part = [x for x in parts if 'desc' in x]
    big_roi_stub = op.basename(roi).split('_')[-3]
    subdivision_rois = [f for f in rois if big_roi_stub in f and ('P' in f or 'M' in f)]
    utils.roi_centers(roi, subdivision_rois, load_img(lgn_ref_vol))

### Look at pRF results

In [ ]:
lgn_rois_prf_space = preproc_layout.get(subject=sub, session=ses_lgn, space='prf', suffix='roi', return_type='file')

In [ ]:
big_lgn_rois_prf_space = [f for f in lgn_rois_prf_space if 'desc-LLGN_' in f or 'desc-RLGN_' in f]
print('\n'.join(lgn_rois_prf_space), big_lgn_rois_prf_space, sep='\n')

In [ ]:
for i, x in zip(range(len(lgn_rois_prf_space)), lgn_rois_prf_space):
    print(i, x)
    utils.roi_stats({i: load_img(x)}, prf_ref_vol_img)

In [ ]:
# prf eccentricity map
prf_ecc_map = "/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_map.nii.gz"
# prf_ecc_map = "/Users/smerdis/data/LGN/BIDS/LL_combined/derivatives/prf/sub-LL/20210908-081328/sub-LL_ses-20210810_desc-prfrho_space-func_thresh-prfrho-12.00_map.nii.gz"

In [ ]:
for roi in big_lgn_rois_prf_space:
    utils.assign_roi_percentile(roi, prf_ecc_map, 33, prf_ref_vol_img,
                                roi_below_suffix='lowecc', roi_above_suffix='highecc')

In [ ]:
for roi in big_lgn_rois_prf_space:
    print(roi)
    parts = op.basename(roi).split('_')
    part = [x for x in parts if 'desc' in x]
    if part:
        big_roi_stub = part[0]
        roi_list = preproc_layout.get(subject=sub, session=ses_lgn, space='prf', suffix='roi', return_type='file')
        subdivision_rois = [f for f in roi_list if big_roi_stub in f and ('ecc' in f)]
        #print(big_roi_stub, subdivision_rois)
        utils.roi_centers(roi, subdivision_rois, load_img(prf_ref_vol_img))

## Coherence analysis to identify LGN voxels responding at flicker frequency

### Use the actual utils.py function

In [ ]:
hemi_bolds, hemi_masks, hemi_eventfiles, hemi_TR, hemi_confounds = utils.get_files(
    sub, ses_lgn, "hemi", raw_data_dir, out_dir,
    run=[1, 2])

### Define hemifield alternation time series etc

In [ ]:
cycle_duration = 27 # (should be 27s)
hemi_freq = (1.0/cycle_duration) # of hemifield alternation, in hertz
n_trs_func = 139 # Length, in TRs, on a functional scan
total_len = n_trs_func*hemi_TR # seconds
Fs = 1/hemi_TR # Sampling freq
trs_exc_beg = 6
trs_exc_end = 1
nfft = 61 #Number of time points to use in the window - 61 gives a band centered at .037037...
#nperseg = n_trs_func - trs_exc_beg - trs_exc_end

# frequency band of physiological relevance
f_lb = 0.02
f_ub = 0.15

# frequency band within which hemifield alternation related activity exists
f_lb_hemi = 0.035
f_ub_hemi = 0.038

print(hemi_freq)

t = np.arange(0.0, total_len, hemi_TR)
t_trim = np.arange((trs_exc_beg-1)*hemi_TR, (total_len - trs_exc_beg - trs_exc_end)+1, hemi_TR)
hemifield_alternation_sinusoid = np.sin(hemi_freq*2*np.pi*t)
hemi_alt_trim = hemifield_alternation_sinusoid[trs_exc_beg:(-1*trs_exc_end)]
print(t, hemifield_alternation_sinusoid.shape)

hemi_ts = ts.TimeSeries(data=hemifield_alternation_sinusoid,
                         sampling_interval=hemi_TR)
hemi_ts_trim = ts.TimeSeries(data=hemi_alt_trim,
                         sampling_interval=hemi_TR)

fig, ax = plt.subplots(1)
ax.set_xlim(0, 50)
ax.plot(t, hemifield_alternation_sinusoid)
#ax.plot(t_trim, hemi_alt_trim)
ax.set_xlabel("Time (sec)")
ax.set_ylabel("Hemifield alternation")

In [ ]:
print(final_big_rois)

In [ ]:
roi = hemi_masks[0]

##### Run 1 of hemifield

In [ ]:
_, lgn_ts_run1 = utils.get_timeseries_from_file(hemi_bolds[0], roi, hemi_TR,
                                detrend=True, standardize='psc', high_pass=f_lb, low_pass=f_ub)

In [ ]:
_, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts, lgn_ts_run1, f_ub, f_lb, dict(NFFT=nfft))

##### Run 2 of hemifield

##### No detrending or standardization

In [ ]:
lgn_masker, lgn_ts_run2 = utils.get_timeseries_from_file(hemi_bolds[1], roi, hemi_TR,
                                detrend=False, standardize=False, high_pass=f_lb, low_pass=f_ub)
_, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts, lgn_ts_run2, f_ub, f_lb, dict(NFFT=nfft))

##### Detrended but no standardization

In [ ]:
lgn_masker, lgn_ts_run2 = utils.get_timeseries_from_file(hemi_bolds[1], roi, hemi_TR,
                                detrend=True, standardize=False, high_pass=f_lb, low_pass=f_ub)
_, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts, lgn_ts_run2, f_ub, f_lb, dict(NFFT=nfft))

##### With detrending and conversion to percent signal change

In [ ]:
lgn_masker, lgn_ts_run2 = utils.get_timeseries_from_file(hemi_bolds[1], roi, hemi_TR,
                                detrend=True, standardize='psc', high_pass=f_lb, low_pass=f_ub)
_, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts, lgn_ts_run2, f_ub, f_lb, dict(NFFT=nfft))

#### Average the runs for better SNR

In [ ]:
lgn_masker

In [ ]:
mean_bold_timeseries = utils.average_timeseries(hemi_bolds, lgn_masker)

In [ ]:
mean_bold_timeseries.shape

In [ ]:
trim_bold_nm = mean_bold_timeseries[trs_exc_beg:(-1*trs_exc_end),:] # note we are using the mean of the two runs
trim_bold_nm.shape

In [ ]:
mean_ts = ts.TimeSeries(data=trim_bold_nm.T, sampling_interval=hemi_TR)
hemi_analyzer, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts_trim, mean_ts, f_ub, f_lb, dict(NFFT=nfft))

##### Run it again but this time with a narrow frequency range for just hemifield alternation

In [ ]:
hemi_analyzer, (hemi_coh_by_voxel, hemi_phase_by_voxel) = utils.seed_coherence_timeseries(hemi_ts_trim, mean_ts,
                                                                   f_ub_hemi, f_lb_hemi, dict(NFFT=nfft))

#### Write coherence values to Nifti file

In [ ]:
lgn_masker

In [ ]:
lgn_masker.get_params()

In [ ]:
len(hemi_coh_by_voxel)

In [ ]:
coherence_img = lgn_masker.inverse_transform(hemi_coh_by_voxel)
coherence_img.shape

In [ ]:
phase_img = lgn_masker.inverse_transform(hemi_phase_by_voxel)
phase_img.shape

In [ ]:
hemi_coh_fn = os.path.join(out_dir, f"sub-{sub}_ses-{ses}_task-hemi_desc-hemialtCoh_map.nii")
hemi_phase_fn = os.path.join(out_dir, f"sub-{sub}_ses-{ses}_task-hemi_desc-hemialtPhase_map.nii")

In [ ]:
coherence_img.to_filename(hemi_coh_fn)
phase_img.to_filename(hemi_phase_fn)

#### Look at cortical ROI coherence with hemifield alternations

In [ ]:
cortical_masker, cortical_ts_run2 = utils.get_timeseries_from_file(hemi_bolds[1], cortical_rois[0], hemi_TR,
                                detrend=False, standardize='zscore', high_pass=f_lb, low_pass=f_ub)
_, (hemi_coh_by_voxel_cortex, hemi_phase_by_voxel_cortex) = utils.seed_coherence_timeseries(hemi_ts, cortical_ts_run2, f_ub_hemi, f_lb_hemi, dict(NFFT=nfft))

#### Write coherence values to Nifti file

In [ ]:
coherence_img = cortical_masker.inverse_transform(hemi_coh_by_voxel_cortex)
coherence_img.shape

In [ ]:
print(cortex_coh_fn)

In [ ]:
coherence_img.to_filename(cortex_coh_fn)

#### SpectralAnalyzer - view and plot FFT, PSD, etc.

In [ ]:
def plot_spectra(S_original, voxid):
    fig01 = plt.figure()
    ax01 = fig01.add_subplot(1, 1, 1)
    if voxid is None:
        ax01.plot(S_original.psd[0],
                  S_original.psd[1],
                  label='PSD')
        ax01.plot(S_original.spectrum_fourier[0],
              np.abs(S_original.spectrum_fourier[1]),
              label='FFT')
#         ax01.plot(S_original.periodogram[0],
#               np.abs(S_original.periodogram[1]),
#               label='Periodogram')

#         ax01.plot(S_original.spectrum_multi_taper[0],
#               np.abs(S_original.spectrum_multi_taper[1]),
#               label='Multitaper')
        
    else:
        ax01.plot(S_original.psd[0],
                  S_original.psd[1][voxid],
                  label='PSD')
        ax01.plot(S_original.spectrum_fourier[0],
              np.abs(S_original.spectrum_fourier[1][voxid]),
              label='FFT')
#         ax01.plot(S_original.spectrum_multi_taper[0],
#               np.abs(S_original.spectrum_multi_taper[1][voxid]),
#               label='Multitaper')
    print(S_original.psd[0], S_original.psd[1], len(S_original.psd[1]), sep='\n')

    ax01.set_xlabel('Frequency (Hz)')
    ax01.set_ylabel('Power')

    ax01.legend()

In [ ]:
roi_mean_timeseries = ts.TimeSeries(np.mean(mean_ts.data, axis=0), sampling_interval=hemi_TR)

In [ ]:
roi_mean_timeseries.data.shape

In [ ]:
plot_spectra(
    nta.SpectralAnalyzer(hemi_ts_trim, method={#'this_method':'welch',
                                                        'NFFT':132,
                                                        'Fs':hemi_ts_trim.sampling_rate})
    , None)
plot_spectra(
    nta.SpectralAnalyzer(roi_mean_timeseries, method={#'this_method':'welch',
                                                        'NFFT':132,
                                                        'Fs':hemi_ts_trim.sampling_rate}), None)

### Mean across voxels of ROI timerseries but with nfft bins

In [ ]:
plot_spectra(
    nta.SpectralAnalyzer(hemi_ts_trim, method={#'this_method':'welch',
                                                        'NFFT':nfft,
                                                        'Fs':hemi_ts_trim.sampling_rate})
    , None)
plot_spectra(
    nta.SpectralAnalyzer(roi_mean_timeseries, method={#'this_method':'welch',
                                                        'NFFT':nfft,
                                                        'Fs':hemi_ts_trim.sampling_rate}), None)

### SeedCoherenceAnalyzer for connectivity

In [ ]:
conn_bolds, conn_masks, conn_eventfiles, conn_TR, conn_confounds = utils.get_files(
    sub, ses_conn, "conn",
    raw_data_dir, out_dir,
    run=[1, 5, 9])

In [ ]:
f_lb = 0.025
f_ub = 0.15

In [ ]:
print("\n".join(big_rois))

In [ ]:
print("\n".join(cortical_rois))
cortical_roi = cortical_rois[0]

In [ ]:
for roi in left_lgn_rois:
    #if 'M' in roi or 'P' in roi:
    bn = op.basename(roi)
    parts = bn.split('_')
    print(parts)
    cohs = []
    phases = []
    for i, bold in enumerate(conn_bolds):
        print(i, bold)
        bbn = op.basename(bold)
        runpart = [p for p in bbn.split('_') if 'run-' in p][0]
        conn_coh_fn = os.path.join(out_dir, f"sub-{sub}_ses-{ses}_task-conn_{runpart}_{parts[-2]}_coherence_map.nii")
        conn_phase_fn = os.path.join(out_dir, f"sub-{sub}_ses-{ses}_task-conn_{runpart}_{parts[-2]}_phase_map.nii")
        print(conn_coh_fn, conn_phase_fn, sep='\n')
        #below: wholebrain mask with mean of seed roi
        # 2/3/21: cahnged to use cortical roi instead of wholebrain
        a, target_masker, coherence_by_voxel, phase_by_voxel = utils.seed_coherence_analysis(bold, cortical_roi, roi,
                          conn_TR, f_ub, f_lb, mean_seed=True, method=dict(NFFT=64))
        coherence_map_img = target_masker.inverse_transform(coherence_by_voxel)
        phase_img = target_masker.inverse_transform(phase_by_voxel)
        coherence_map_img.to_filename(conn_coh_fn)
        phase_img.to_filename(conn_phase_fn)
        # below: roi with the mean of that roi
        #a = utils.seed_coherence_analysis(bold, roi, roi,
        #      conn_TR, f_ub, f_lb, conn_coh_fn, mean_seed=True)
        # below: cortical roi (hand-defined and hardcoded) with mean of seed ROI
#         a, _ = utils.seed_coherence_analysis(bold, cortical_rois[0], roi,
#               conn_TR, f_ub, f_lb, conn_coh_fn, mean_seed=True)

        cohs.append(a.coherence)
        phases.append(a.relative_phases)    

In [ ]:
img1 = '/Users/smerdis/data/LGN/BIDS/NB_2020/derivatives/streams/sub-NB/sub-NB_ses-20201029_task-conn_run-01_desc-LLGNsmallM80_phase_map.nii'
img2 = '/Users/smerdis/data/LGN/BIDS/NB_2020/derivatives/streams/sub-NB/sub-NB_ses-20201029_task-conn_run-05_desc-LLGNsmallM80_phase_map.nii'


In [ ]:
phase_diff_img = math_img(f"img1 - img2", img1=img1, img2=img2)
phase_diff_path = f"{out_dir}/sub-NB_ses-20201029_task-conn_desc-LLGNsmallM80_leftM_phasediff_map.nii"
phase_diff_img.to_filename(phase_diff_path)

In [ ]:
print(f"fsleyes {conn_ref_vol_path} {phase_diff_path}")

In [ ]:
cohs[0].shape